<a href="https://colab.research.google.com/github/tjdrhks0808/Seminar_in_Statistics/blob/gh-pages/KcBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Last Update @ 2020.12.04

- Huggingface Transformers 4.0.0  버전 반영

# 패키지 import & 기본 Args 설정

In [ ]:
import os
import pandas as pd

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import BertForSequenceClassification, BertTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
# import emoji
from soynlp.normalizer import repeat_normalize

# Package 설치 & 데이터 받기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers pytorch_lightning emoji soynlp

     |████████████████████████████████| 2.6 MB 35.7 MB/s 
     |████████████████████████████████| 813 kB 50.6 MB/s 
     |████████████████████████████████| 185 kB 13.2 MB/s 
     |████████████████████████████████| 416 kB 49.0 MB/s 
     |████████████████████████████████| 895 kB 42.3 MB/s 
     |████████████████████████████████| 3.3 MB 45.4 MB/s 
     |████████████████████████████████| 636 kB 47.7 MB/s 
     |████████████████████████████████| 118 kB 67.6 MB/s 
     |████████████████████████████████| 234 kB 73.5 MB/s 
     |████████████████████████████████| 829 kB 44.1 MB/s 
     |████████████████████████████████| 10.6 MB 35.0 MB/s 
     |████████████████████████████████| 1.3 MB 34.2 MB/s 
     |████████████████████████████████| 294 kB 71.8 MB/s 
     |████████████████████████████████| 142 kB 71.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLP/데이터_정제.csv')
raw = pd.read_excel('/content/drive/MyDrive/NLP/공모전_제공_데이터(1차).xlsx')

In [ ]:
raw_segment = []
for i in range(1000):
  raw_segment.append(raw.iloc[:,-1][i][:2])

In [ ]:
neg_idx=[]
pos_idx=[]
for i in range(1000):
  if raw_segment[i] == '불만':
    neg_idx.append(i)
  else:
    pos_idx.append(i)

In [ ]:
label = [0 for i in range(1000)]
for i in neg_idx:
  label[i] = 1

In [ ]:
df['label'] = label

In [ ]:
df = df[ ['네이버', 'label'] ]

In [ ]:
df[514:517]

,네이버,label
514,카드 사용 시 포인트 적립이 적다고 생각됨,1
515,많은 포인트 혜택을 주세요,1
516,보이는 화면,0


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.33, random_state=42)

## 기본 학습 Arguments

In [ ]:
class Arg:
    random_seed: int = 42  # Random Seed
    pretrained_model: str = 'beomi/kcbert-base'  # Transformers PLM name
    pretrained_tokenizer: str = ''  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    auto_batch_size: str = 'power'  # Let PyTorch Lightening find the best batch size 
    batch_size: int = 0  # Optional, Train/Eval Batch Size. Overrides `auto_batch_size` 
    lr: float = 5e-6  # Starting Learning Rate
    epochs: int = 20  # Max Epochs
    max_length: int = 150  # Max Length input size
    report_cycle: int = 100  # Report (Train Metrics) Cycle
    cpu_workers: int = os.cpu_count()  # Multi cpu workers
    test_mode: bool = False  # Test Mode enables `fast_dev_run`
    optimizer: str = 'AdamW'  # AdamW vs AdamP
    lr_scheduler: str = 'exp'  # ExponentialLR vs CosineAnnealingWarmRestarts
    fp16: bool = False  # Enable train on FP16
    tpu_cores: int = 0  # Enable TPU with 1 core or 8 cores

args = Arg()

## 기본값을 Override 하고싶은 경우 아래와 같이 수정

In [ ]:
!nvidia-smi

Mon Jul 26 00:18:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

위에서 GPU가 V100/P100이면 아래 `batch_size`  를 32 이상으로 하셔도 됩니다.

In [ ]:
# args.tpu_cores = 8  # Enables TPU
args.fp16 = True  # Enables GPU FP16
args.batch_size = 32  # Force setup batch_size

# Model 만들기 with Pytorch Lightning

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
            args.pretrained_tokenizer
            if args.pretrained_tokenizer
            else args.pretrained_model
        )

In [ ]:
class Model(LightningModule):
    def __init__(self, options):
        super().__init__()
        self.args = options
        self.bert = BertForSequenceClassification.from_pretrained(self.args.pretrained_model)
        self.tokenizer = BertTokenizer.from_pretrained(
            self.args.pretrained_tokenizer
            if self.args.pretrained_tokenizer
            else self.args.pretrained_model
        )

    def forward(self, **kwargs):
        return self.bert(**kwargs)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits
        
        preds = logits.argmax(dim=-1)

        y_true = labels.cpu().numpy()
        y_pred = preds.cpu().numpy()

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'train_loss': loss.cpu().detach().numpy().tolist(),
            'train_acc': metrics[0],
            'train_precision': metrics[1],
            'train_recall': metrics[2],
            'train_f1': metrics[3],
        }
        if (batch_idx % self.args.report_cycle) == 0:
            print()
            pprint(tensorboard_logs)
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }

    def validation_epoch_end(self, outputs):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        _loss = loss / len(outputs)

        loss = float(_loss)
        y_true = []
        y_pred = []

        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'val_loss': loss,
            'val_acc': metrics[0],
            'val_precision': metrics[1],
            'val_recall': metrics[2],
            'val_f1': metrics[3],
        }

        print()
        pprint(tensorboard_logs)
        return {'loss': _loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        if self.args.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.args.lr)
        elif self.args.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.args.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.args.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.args.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def read_data(self, path):
        if path.endswith('xlsx'):
            return pd.read_excel(path)
        elif path.endswith('csv'):
            return pd.read_csv(path)
        elif path.endswith('tsv') or path.endswith('txt'):
            return pd.read_csv(path, sep='\t')
        else:
            raise NotImplementedError('Only Excel(xlsx)/Csv/Tsv(txt) are Supported')

    def preprocess_dataframe(self, df):
        pattern = re.compile(f'[^ .,?!~·∼()\x00-\x7Fㄱ-힣]+')

        def clean(x):
            x = pattern.sub(' ', x)
            x = x.strip()
            x = repeat_normalize(x, num_repeats=2)
            return x

        df['네이버'] = df['네이버'].map(lambda x: self.tokenizer.encode(
            str(x),
            padding='max_length',
            max_length=self.args.max_length,
            truncation=True,
        ))
        return df

    def train_dataloader(self):
        df = train
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['네이버'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=True,
            num_workers=self.args.cpu_workers,
        )

    def val_dataloader(self):
        df = test
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['네이버'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=False,
            num_workers=self.args.cpu_workers,
        )


In [ ]:
def main():
    print("Using PyTorch Ver", torch.__version__)
    print("Fix Seed:", args.random_seed)
    seed_everything(args.random_seed)
    model = Model(args)

    print(":: Start Training ::")
    trainer = Trainer(
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        auto_scale_batch_size=args.auto_batch_size if args.auto_batch_size and not args.batch_size else False,
        # For GPU Setup
        deterministic=torch.cuda.is_available(),
        gpus=-1 if torch.cuda.is_available() else None,
        precision=16 if args.fp16 else 32,
        # For TPU Setup
        # tpu_cores=args.tpu_cores if args.tpu_cores else None,
    )
    trainer.fit(model)

# 학습!

> 주의: 1epoch별로 GPU-P100기준 약 1-2시간, GPU V100기준 ~30분이 걸립니다.

> Update @ 2020.09.01
> 최근 Colab Pro에서 V100이 배정됩니다.

```python
# 1epoch 기준 아래 score가 나옵니다.
{'val_acc': 0.90522,
 'val_f1': 0.9049023739289227,
 'val_loss': 0.23429009318351746,
 'val_precision': 0.9143146796431468,
 'val_recall': 0.8956818813808446}
```

In [ ]:
main()

Global seed set to 42


Using PyTorch Ver 1.9.0+cu102
Fix Seed: 42


Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

:: Start Training ::


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...


{'val_acc': 0.48787878787878786,
 'val_f1': 0.0,
 'val_loss': 0.9308958649635315,
 'val_precision': 0.0,
 'val_recall': 0.0}

{'train_acc': 0.90625,
 'train_f1': 0.9230769230769231,
 'train_loss': 0.2080157995223999,
 'train_precision': 0.9,
 'train_recall': 0.9473684210526315}



{'val_acc': 0.48787878787878786,
 'val_f1': 0.0,
 'val_loss': 0.9387996196746826,
 'val_precision': 0.0,
 'val_recall': 0.0}

{'train_acc': 0.9375,
 'train_f1': 0.9375,
 'train_loss': 0.12427937984466553,
 'train_precision': 0.9375,
 'train_recall': 0.9375}



{'val_acc': 0.48787878787878786,
 'val_f1': 0.0,
 'val_loss': 0.9919024109840393,
 'val_precision': 0.0,
 'val_recall': 0.0}

{'train_acc': 0.90625,
 'train_f1': 0.9302325581395349,
 'train_loss': 0.22188282012939453,
 'train_precision': 1.0,
 'train_recall': 0.8695652173913043}



{'val_acc': 0.48787878787878786,
 'val_f1': 0.0,
 'val_loss': 1.0015681982040405,
 'val_precision': 0.0,
 'val_recall': 0.0}

{'train_acc': 0.96875,
 'train_f1': 0.9714285714285714,
 'train_loss': 0.08992493152618408,
 'train_precision': 0.9444444444444444,
 'train_recall': 1.0}



{'val_acc': 0.48787878787878786,
 'val_f1': 0.0,
 'val_loss': 1.0466152429580688,
 'val_precision': 0.0,
 'val_recall': 0.0}

